In [7]:
from json_extractor import from_file_get_n_docs
from collections import defaultdict
from tagged_document_parsers import parse_stanford_doc
import json

# Read in Data

In [8]:
stanford_file_name = './50_tagged_by_stanford.jsonl'
wikifier_file_name = './100_tagged_by_wikifier.jsonl'
tagme_file_name = './100_tagged_by_tagme_longtext_0_epsilon_dot1_includecategories_includeallspots.jsonl'

In [9]:
stanford_docs = from_file_get_n_docs(stanford_file_name, 10)

In [10]:
wikifier_docs = from_file_get_n_docs(wikifier_file_name, 10)

In [11]:
tagme_docs = from_file_get_n_docs(tagme_file_name, 10)

# Stanford to Mention Dict

In [12]:
stanford_parsed_docs = [parse_stanford_doc(doc) for doc in stanford_docs]